In [47]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import webdrivermanager

# 1. setup 
driver = webdriver.Chrome()
dict_female = dict()
dict_male = dict()


# 2. standard procedure for each page

# 2a. get available urls to sub pages for a category/culture
def get_all_urls(url):
    all_names = list()
    urls = list()
    urls.append(url)
    #print("STEP 3: " + url)

    driver.get(url)
    table_anchors = driver.find_elements_by_xpath("//table/tbody/tr/td[@bordercolor='#000000'][@valign='top']/p[@align='center']/font[@size='1'][@color='#FFFFFF']/a")
    #table_anchors = driver.find_elements_by_xpath("//table[@width='100%']/tbody/tr[6]/td[@valign='top']/p[@align='center']/font/a")
    urls.extend(o.get_attribute('href') for o in table_anchors) 
    
    if(len(urls) >= 3):
        urls.pop(1) # 'remove contact'
        urls.pop(1) # remove 'male names' links

  
    for url in urls:
        #print(url)
        names = get_names_by_page(url)
        all_names.extend(names)

        # debug
        txt = "Url: {url} - {len}"
        print(txt.format(url = url, len = len(all_names)))
        
        if(len(all_names) >= 900):
            break


    return all_names

# 2b. extract names by page and save them in list
def get_names_by_page(url):
    names = list()
    driver.get(url)

    table_with_names = driver.find_elements_by_xpath("//table/tbody/tr/td//ol/li/font/a")   #/a
  
    attr = (o.text for o in table_with_names) 
    names.extend(attr)

    return names      


# fill dictionary with all names by culture and gender
def fill_names_dict(url, gender):
    dict_out = dict()

    driver.get(url)
    categories_cultures = driver.find_elements_by_xpath("//td[@width='46%']/table/tbody/tr/td/p")

    for x in range(1, len(categories_cultures)):
        culture = categories_cultures[x]
        print(culture)

        # get URLs and save them
        elements = culture.find_elements_by_tag_name("a")
  
        culture_key = elements[0].text # get text of first <a>  
        if(len(elements) >= 3):
            link = elements[gender].get_attribute('href')  # get desired url  
            dict_out[culture_key] = link  


    # get names of each page
    for culture_key, val in dict_out.items():
        dict_out[culture_key] = get_all_urls(val)
   
    return dict_out

In [15]:
# 3. implementation 

# get all female names by culture
url = "http://www.20000-names.com"
dict_female = fill_names_dict(url, 0) # 0:female, 1:male

# end web scrapping
driver.quit()

# 4a output
print(dict_female.keys())

<selenium.webdriver.remote.webelement.WebElement (session="5185004ee61a34b22d89d3d8213d9df1", element="470360a6-0bea-43aa-a7a5-19da2b476062")>
<selenium.webdriver.remote.webelement.WebElement (session="5185004ee61a34b22d89d3d8213d9df1", element="d2794a28-aaf3-491b-a5b7-08414522c5d3")>
<selenium.webdriver.remote.webelement.WebElement (session="5185004ee61a34b22d89d3d8213d9df1", element="1202c21e-c888-4759-91bd-6fb2a64dc4c4")>
<selenium.webdriver.remote.webelement.WebElement (session="5185004ee61a34b22d89d3d8213d9df1", element="224e8e00-2667-4706-80cd-3bf26a0eab59")>
<selenium.webdriver.remote.webelement.WebElement (session="5185004ee61a34b22d89d3d8213d9df1", element="b7eb4f33-d0dc-4aa8-a7ac-aa6e6e2df4de")>
<selenium.webdriver.remote.webelement.WebElement (session="5185004ee61a34b22d89d3d8213d9df1", element="3d8786a9-9a8f-4499-a98c-8608cb5eac2f")>
<selenium.webdriver.remote.webelement.WebElement (session="5185004ee61a34b22d89d3d8213d9df1", element="321f0881-3efa-4dd9-85b5-e61e9fc499af")>

In [17]:
print(dict_female)                             

In [48]:
# 3. implementation 

# get all male names by culture
url = "http://www.20000-names.com"
dict_male = fill_names_dict(url, 1)  # 0:female, 1:male

# end web scrapping
driver.quit()

# 4b output
print(dict_male.keys())

<selenium.webdriver.remote.webelement.WebElement (session="2f5d5b19675fb70dfa837ccebf76ec2a", element="2397c93d-e45f-4c6c-a5c2-7356d314de0d")>
<selenium.webdriver.remote.webelement.WebElement (session="2f5d5b19675fb70dfa837ccebf76ec2a", element="ccb67d66-2b25-4085-8d3e-d3b74afcbb35")>
<selenium.webdriver.remote.webelement.WebElement (session="2f5d5b19675fb70dfa837ccebf76ec2a", element="5e4e12f2-ca02-4b49-9401-476488b315cd")>
<selenium.webdriver.remote.webelement.WebElement (session="2f5d5b19675fb70dfa837ccebf76ec2a", element="b9d55ee7-f88d-4cfd-be43-917b1a11cc21")>
<selenium.webdriver.remote.webelement.WebElement (session="2f5d5b19675fb70dfa837ccebf76ec2a", element="e2a94696-0923-4c89-839d-e976d174b3d5")>
<selenium.webdriver.remote.webelement.WebElement (session="2f5d5b19675fb70dfa837ccebf76ec2a", element="39069c31-d2f5-4975-aeff-95377c4b978d")>
<selenium.webdriver.remote.webelement.WebElement (session="2f5d5b19675fb70dfa837ccebf76ec2a", element="be1df4ee-04f2-4055-8164-b0f472099c79")>

In [ ]:
print(dict_male)

In [55]:
# 5. dataframe preparation
import pandas as pd

# create dataframes from dictionaries

# convert original dict to tuple, convert each v to Series type (to set empty columns to NaN) and set k to key of new dictionary
df_female = pd.DataFrame(dict([(k, pd.Series(v)) for k,v in dict_female.items()]))
df_male = pd.DataFrame(dict([(k, pd.Series(v)) for k,v in dict_male.items()]))

<ipython-input-55-7155e4936f4c>:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_males = pd.DataFrame(dict([(k, pd.Series(v)) for k,v in dict_male.items()]))


In [59]:
# shape
print(df_female.shape)
print(df_male.shape)

(0, 0)
(996, 68)


""


In [60]:
# data cleaning and setting for merge
import re

# a. only keep the culture(s)
df_female.columns = df_female.columns.str.replace('(\s*)Names(\s*)', '', regex = True)
df_male.columns = df_male.columns.str.replace('(\s*)Names(\s*)', '', regex = True)

In [61]:
# capitalize all cells
df_female = df_female.applymap(lambda s:s.capitalize() if type(s) == str else s)
df_male = df_male.applymap(lambda s:s.capitalize() if type(s) == str else s)

In [62]:
df_male

,African,Akkadian,Albanian,Anglo-Saxon,Arabian,Aramaic,Armenian,Arthurian Legend,Assyrian,"Aztec, Nahuatl",...,Swedish,Swiss,Teutonic,"Thai, Siamese",Turkish,Ukrainian,Vietnamese,Welsh,Yiddish,Special Categories:
0,Abebe,Abdi-ili,Agim,Cenred,Aali,Abaya,Abirad,Afallach,Khani,NaN,...,Bo,NaN,Þórr,NaN,Adam,Olek,Dai,Agrona,Yitzchak,Nova
1,Abel,Abilkisu,Agon,Cenric,Abbas,Abaye,Agatangeghos,Afallech,Khar-si-yesu,NaN,...,Bjorg,NaN,Manu,NaN,Ahmed,Oleksander,Hai,Bran,Asher,NaN
2,Abidemi,Abi-maras,Aleksandër,Ceolsige,Abd,Abayi,Anania,Accalon,Khasis-adra,NaN,...,Bo,NaN,NaN,NaN,Baatar,NaN,Quan,Brennus,Aharown,NaN
3,Abimbola,Abil-ili,Andrea,Cerdic,Abdal,Abba,Anastas,Accolon,Khassikhu,NaN,...,Birger,NaN,NaN,NaN,Iskender,NaN,Tu,Brenin,Fishel,NaN
4,Abioye,Abil-ilishu,Arben,Chad,Abdallah,Abdima,Andon,Addanc,Khumri,NaN,...,Knútr,NaN,NaN,NaN,Alexandrus,NaN,NaN,Llwyd,Gabriyel,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# export as Excel files
df_female.to_excel("names_culture_female.xlsx")
df_male.to_excel("names_culture_male.xlsx")